In [13]:
import pandas as pd
from zipfile import ZipFile

In [14]:
zip = ZipFile('files/data.zip')
zip.extractall()

IOError: [Errno 2] No such file or directory: 'files/data.zip'

In [3]:
ls

data.zip        example.json    example_app.py
